<a href="https://colab.research.google.com/github/FajarKKP/ML-DL_related_project/blob/main/SIBI_Transfer_Learning_EfficientNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# import os
# import tensorflow as tf
# from tensorflow import keras
# from tensorflow.keras import layers
# from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
import tensorflow as tf
import keras
import matplotlib.pyplot as plt

from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [3]:
# Install library that can help split dataset in folder
# pip install split-folders tqdm


Preprocess Dataset

In [4]:
#Import data from Gdrive
data_dir = '/content/drive/MyDrive/Dataset_pakai'

# Initialize batch size and image size
batch_size = 32
image_size = (224,224)

In [5]:
# import splitfolders
# splitfolders.ratio(data_dir, output="/content/drive/MyDrive/testing_folder", seed=1337, ratio=(.8, .1, .1), group_prefix=None, move=False)

In [6]:
# # Get data from gdrive. Split and resize 

# train_ds = tf.keras.preprocessing.image_dataset_from_directory(
#     "/content/drive/MyDrive/testing_folder/train",
#     labels = 'inferred',
#     validation_split=0.2,
#     subset="training",
#     shuffle = True,
#     seed=1337,
#     image_size=image_size,
#     batch_size=batch_size,
# )

# val_ds = tf.keras.preprocessing.image_dataset_from_directory(
#     "/content/drive/MyDrive/testing_folder/val",
#     labels = 'inferred',
#     validation_split=0.2,
#     subset="validation",
#     seed=1337,
#     image_size=image_size,
#     batch_size=batch_size,
# )



In [7]:
# For Augmentation
train_datagen = ImageDataGenerator(
                    rescale=1./255,
                    rotation_range=10,
                    horizontal_flip=True)
 
test_datagen = ImageDataGenerator(rescale=1./255,)

In [16]:

train_generator = train_datagen.flow_from_directory(
        '/content/drive/MyDrive/testing_folder/train', 
        target_size=image_size, 
        batch_size=batch_size,
        # shuffle = True,
        class_mode='sparse')
 
validation_generator = test_datagen.flow_from_directory(
        '/content/drive/MyDrive/testing_folder/val', 
        target_size=image_size, 
        batch_size=batch_size,
        # shuffle = True,
        class_mode='sparse')

test_generator = test_datagen.flow_from_directory(
        '/content/drive/MyDrive/testing_folder/test', 
        target_size=image_size, 
        batch_size=batch_size,
        class_mode='sparse')

Found 9600 images belonging to 24 classes.
Found 1200 images belonging to 24 classes.
Found 1200 images belonging to 24 classes.


In [9]:
# # Input Data Augmentation

# data_augmentation = keras.Sequential(
#     [
#         layers.RandomFlip("horizontal"),
#         layers.RandomRotation(0.1)
#         # layers.Rescaling(scale = 1./255)
#     ]
# )

In [10]:
# # Augment the train and validation dataset
# augmented_train_ds = train_ds.map(
#   lambda x, y: (data_augmentation(x, training=True), y))

# augmented_val_ds = val_ds.map(
#   lambda x, y: (data_augmentation(x, training=True), y))

In [11]:
# # Example of dataset that has been augmented
# plt.figure(figsize=(10, 10))
# for images, labels in augmented_train_ds.take(1):
#     for i in range(9):
#         ax = plt.subplot(3, 3, i + 1)
#         plt.imshow(images[i].numpy().astype("uint8"))
#         plt.title(int(labels[i]))
#         plt.axis("off")

The Model

In [12]:
# Import the EfficientNet model
eff_model1 = tf.keras.applications.efficientnet.EfficientNetB0(include_top=False, input_shape=(224,224,3))
print(type(eff_model1))
eff_model1.summary()

<class 'keras.engine.functional.Functional'>
Model: "efficientnetb0"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 rescaling (Rescaling)          (None, 224, 224, 3)  0           ['input_1[0][0]']                
                                                                                                  
 normalization (Normalization)  (None, 224, 224, 3)  7           ['rescaling[0][0]']              
                                                                                                  
 stem_conv_pad (ZeroPadding2D)  (None, 2

In [13]:
# Freeze the EfficientNet, then set the dropout rate we will use for our model
eff_model1.trainable = False
dropout_rate = 0.1 #@param {type:"slider", min:0, max:1, step:0.1}

In [14]:
# Create a new model, combining the EfficientNet and new output layer with 24 output class

model = models.Sequential()
model.add(eff_model1)
model.add(layers.GlobalMaxPooling2D(name="gap"))
# model.add(layers.Flatten(name="flatten"))
if dropout_rate > 0:
    model.add(layers.Dropout(dropout_rate, name="dropout_out"))

model.add(layers.Dense(24, activation="softmax", name="fc_out"))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 efficientnetb0 (Functional)  (None, 7, 7, 1280)       4049571   
                                                                 
 gap (GlobalMaxPooling2D)    (None, 1280)              0         
                                                                 
 dropout_out (Dropout)       (None, 1280)              0         
                                                                 
 fc_out (Dense)              (None, 24)                30744     
                                                                 
Total params: 4,080,315
Trainable params: 30,744
Non-trainable params: 4,049,571
_________________________________________________________________


In [ ]:
# Train model
epochs = 50

class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epochs, logs={}):
    if(logs.get('accuracy')>0.975):
      print("\n Good enough, its above 98%")
      self.model.stop_training=True
callbacks = myCallback()

# early_stopping = keras.callbacks.EarlyStopping(
#     monitor="accuracy",
#     patience=5,
  
# )

model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-3),
    loss="sparse_categorical_crossentropy", #"sparse_categorical_crossentropy" if label is in integer
    metrics=["accuracy"],
)
final_model = model.fit(
    train_generator, epochs=epochs, callbacks=[callbacks], validation_data=validation_generator,
)

Epoch 1/50
  8/300 [..............................] - ETA: 8:20 - loss: 8.1073 - accuracy: 0.0430

In [ ]:
def plot_hist(final_model):
    plt.plot(final_model.history["accuracy"])
    plt.plot(final_model.history["val_accuracy"])
    plt.title("model accuracy")
    plt.ylabel("accuracy")
    plt.xlabel("epoch")
    plt.legend(["train", "validation"], loc="upper left")
    plt.show()


plot_hist(final_model)

In [ ]:
# Confusion Matrix

In [ ]:
model.save('/content/drive/MyDrive/Hasil_model_EfficientNet/final_modelv2.h5')
model.save('/content/drive/MyDrive/Hasil_model_EfficientNet/final_modelv2.hdf5')